In [1]:
import pycurl
from bs4 import BeautifulSoup
from io import BytesIO
import re
import certifi
import random

In [84]:
# класс страниц
class top:
    def __init__(self, value = "", url = "",):
        self.value = value
        self.url = url
        self.link_list = [] # список ссылок
        self.rank = 0

    def get_page(self, new_url): # создание страницы
        self.url = new_url
        self.value = re.split('/', new_url)[-1]
        
        # открытие страницы и копирование html-кода
        buffer = BytesIO()
        c = pycurl.Curl()
        c.setopt(c.URL, new_url)
        c.setopt(c.WRITEDATA, buffer)
        c.setopt(c.CAINFO, certifi.where())
        c.perform()
        c.close()
        hrefs = 0
        self.link_list = []
        
        body = buffer.getvalue() # получаем html-код
        soap = BeautifulSoup(body.decode('utf-8')) # декодируем
        soap = soap.find("div", {"class" : "mw-parser-output"}) # берём только нудный нам div
        # создаём списки ссылок, которые нам не нужны
        delete = soap.findAll("a",attrs={'href': [re.compile("jpg"), re.compile("png"), re.compile("Help:"),
                                                     re.compile("wikidata"),
                                                     re.compile("Template:"),
                                                     re.compile("Template_talk:"),
                                                     re.compile("Wikipedia:"),
                                                     re.compile("User:"),
                                                     re.compile("User_talk:"),
                                                     re.compile("Talk:")]})
        delete_2 = soap.findAll("a",attrs={'class' : re.compile("")})
        delete_3 = soap.findAll("a",attrs={'title' : [re.compile("Special:"), re.compile("Digital object identifier"), 
                                                      re.compile("International Standard Book Number"),
                                                     re.compile("International Standard Serial Number")]})
        delete = list(set(delete+delete_2+delete_3))
        
        # список всех ссылок
        hrefs = soap.findAll('a', attrs={'href': re.compile("/wiki/")}) 
        # удаление ненужных ссылок
        hrefs = list(set([x for x in hrefs if (x not in delete)]))
        # добавляем начальную часть к ссылкам
        for i in hrefs:
            self.link_list.append("https://en.wikipedia.org"+re.split("#", i['href'])[0])
        
        

    def rank_up(self):
        self.rank += 1

    def __str__ (self):
        to_print = ""
        to_print += (self.value) + "\n"
        to_print += (self.url) + "\n"
        for i in self.link_list:
            to_print += (i) + "\n"
        return to_print

# класс графа
class graph:
    def __init__(self):
        self.list_of_tops = []
    # добавляем страницу в граф
    def add_top(self, url):
        # проверка, есть ли страница уже в графе
        flag = 0
        for i in self.list_of_tops:
            if url == i.url:
                flag = 1
                break
        
        if flag != 1:
            new_top = top()
            new_top.get_page(url)
            self.list_of_tops.append(new_top)

    # straight определение ранка страницы
    def update_ranks(self):
        for i in self.list_of_tops:
            #print(i.url)
            for j in self.list_of_tops:
                if i.url in j.link_list and i.url != j.url:
                    #print(i.rank)
                    i.rank_up()

    def __str__ (self):
        to_print = ""
        for i in self.list_of_tops:
            to_print += i.value + "   " + str(i.rank) + "\n"
        return to_print    

In [85]:
graph1 = graph()
graph1.add_top("https://en.wikipedia.org/wiki/Human_brain")
c = 0
for i in range(500):
    c += 1
    if c%100 == 0:
        print(c)
    graph1.add_top(random.choice((random.choice(graph1.list_of_tops)).link_list))
graph1.update_ranks()

100
200
300
400
500


In [86]:
print(graph1)

Human_brain   7
Anatomical_terms_of_location   6
Orchiectomy   8
Physiognomy   2
Reflection_symmetry   1
Dog   27
Stonehenge   27
Somatotype_and_constitutional_psychology   1
Urocyon   20
Carreg_Samson   21
Northern_elephant_seal   20
OCLC   69
George_Borrow   2
Mesoderm   22
Ard_(plough)   6
Epiblast   16
Clonmel   16
Inguinal_orchiectomy   1
Travel_literature   11
Ormond_Lower   9
The_New_York_Times   35
Vasectomy   6
Reproductive_system   14
Thurles   11
Angular_resolution   1
Bryn_Gwyn_stones   22
Mirror   6
Encyclop%C3%A6dia_Britannica_Eleventh_Edition   30
Ikerrin   10
Coitus_interruptus   1
Bu%C4%A1ibba_Temple   21
Phaedo_of_Elis   7
Neuromere   5
Trilaminar_blastocyst   15
Boskednan_stone_circle   21
Global_Biodiversity_Information_Facility   41
Foramen_magnum   8
The_Sorrows_of_Young_Werther   6
Killenaule   7
Eel_life_history   3
McFarland_%26_Company   1
Primitive_pit   15
Stoicism   21
Sophist   11
Horse_and_Jockey   7
Athenaeus   2
Irish_language   37
Himerius   1
Bilateri

In [ ]:






+.